In [ ]:
# This is an old config file
## Most of this is not relevant
## The only things that are relevant are those related to dataloading
## I will try to delete most of the irrelvant stuff

from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
NUM_CHANNELS = 16  # Note that ELEC573Net is hardcoded in, doesn't use this...
NUM_TRAIN_GESTURES = 8  # For pretrained models
NUM_FT_GESTURES = 1  # Oneshot finetuning

config = {}

config["feature_engr"] = "None"  # TODO: Is this up to date? Idk if this is relevant to meta-aug
config["time_steps"] = 1  # TODO: Is this up to date? Idk if this is relevant to meta-aug
config["sequence_length"] = 64  # TODO: Is this up to date? Idk if this is relevant to meta-aug
config["num_train_gesture_trials"] = 9  # TODO: Is this up to date? Idk if this is relevant to meta-aug
config["num_ft_gesture_trials"] = 1  # TODO: Is this up to date? Idk if this is relevant to meta-aug
config["num_pretrain_users"] = 24  # TODO: Is this up to date? Idk if this is relevant to meta-aug
config["num_testft_users"] = 4  # TODO: Is this up to date? Idk if this is relevant to meta-aug
config["timestamp"] = timestamp
config["num_classes"] = 10  # TODO: Is this up to date? Idk if this is relevant to meta-aug
config["verbose"] = False
config["num_total_users"] = 32

config["multimodal"] = True
config['emg_in_ch'] = 16
config['imu_in_ch'] = 72
config['demo_in_dim'] = 12

# ADDING MAML SPECIFIC -- this stuff is not yet relevant to the meat aug project
config["meta_learning"] = False
config["n_way"] = 10
config["k_shot"] = 1
config["q_query"] = 9  # TODO: Does this need to be 9? If it set it lower does that just make it faster? Does that impact the model? Slightly noiser eval??

# These are my data paths, they need to be updated individually unless we want to do relative paths
config["emg_imu_pkl_full_path"] = 'C:\\Users\\kdmen\\Box\\Yamagami Lab\\Data\\Meta_Gesture_Project\\filtered_datasets\\metadata_IMU_EMG_allgestures_allusers.pkl'
config["pwmd_xlsx_filepath"] = "C:\\Users\\kdmen\\Repos\\fl-gestures\\Biosignal gesture questionnaire for participants with disabilities.xlsx"
config["pwoutmd_xlsx_filepath"] = "C:\\Users\\kdmen\\Repos\\fl-gestures\\Biosignal gesture questionnaire for participants without disabilities.xlsx"
config["dfs_save_path"] = "C:\\Users\\kdmen\\Repos\\fl-gestures\\April_25\\MOE\\full_datasplit_dfs\\"
config["dfs_load_path"] = "C:\\Users\\kdmen\\Repos\\fl-gestures\\April_25\\MOE\\full_datasplit_dfs\\Initial_Multimodal\\"
#config["saved_df_timestamp"] = '20250917_1217'
# These need to get set by the json split right... are those set somewhere... --> THESE ARE HARDCODED IN load_multimodal_data_loaders right now!!
#train_PIDs=['P104', 'P105', 'P106', 'P107', 'P108', 'P109', 'P112', 'P114', 'P115', 'P116', 'P118', 'P119', 'P123', 'P124', 'P125', 'P126', 'P127', 'P128', 'P004', 'P005', 'P006', 'P008', 'P010', 'P011'], 
#val_PIDs=['P102', 'P110', 'P121', 'P131'], 
#test_PIDs=['P103', 'P111', 'P122', 'P132'], 

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
import pandas as pd
import pickle


# TODO: This needs to be updated to use config instead of default values...
## And to accomodate k-fold cross validation...
def load_multimodal_datasets(config, use_emg=True, use_imu=True, use_demographics=True, load_existing_dfs=False, save_dfs=False):

    if use_emg is False or use_imu is False or use_demographics is False:
        raise ValueError("Currently, emg, imu, and demographics all must be used.")

    emg_imu_pkl_full_path = config["emg_imu_pkl_full_path"]
    pwmd_xlsx_filepath = config["pwmd_xlsx_filepath"]
    pwoutmd_xlsx_filepath = config["pwoutmd_xlsx_filepath"]
    dfs_save_path = config["dfs_save_path"]
    dfs_load_path = config["dfs_load_path"]
    saved_df_timestamp = config["saved_df_timestamp"]
    
    # This is the user split... should be set according to the json? Double check...
    train_PIDs = config["train_PIDs"]
    val_PIDs = config["val_PIDs"]
    test_PIDs = config["test_PIDs"]
    train_gesture_range = config["train_gesture_range"] 
    valtest_gesture_range = config["valtest_gesture_range"]

    # ----------------
    # Build / load DataFrames
    # ----------------
    if load_existing_dfs is False:
        data_df = pd.read_pickle(emg_imu_pkl_full_path)

        metadata_cols = ['Participant', 'Gesture_ID', 'Gesture_Num']
        metadata_cols_df = data_df[metadata_cols].rename(columns={"Participant": "PID"})
        metadata_cols_df['Gesture_Num'] = metadata_cols_df['Gesture_Num'].astype(int)

        # PID encoder
        all_PIDs = metadata_cols_df['PID']
        unique_PIDs = all_PIDs.unique()
        PID_encoder = LabelEncoder().fit(unique_PIDs)

        # Gesture encoder
        gesture_ID_label_encoder = LabelEncoder()
        metadata_cols_df['Enc_Gesture_ID'] = gesture_ID_label_encoder.fit_transform(metadata_cols_df['Gesture_ID'])
        metadata_cols_df['Enc_PID'] = PID_encoder.transform(metadata_cols_df['PID'])

        # Signals
        X_df = data_df.drop(metadata_cols, axis=1)
        ppd_B_X_df = preprocess_df_B_by_gesture(X_df)

        # Demographics (with & without disabilities)
        FULL_pwmd_demo_df = pd.read_excel(pwmd_xlsx_filepath)
        pwmd_demo_df = FULL_pwmd_demo_df[[
            "PID", "disability coding", "time disabled", "Actual handedness",
            "What is your age?", "What is your gender?", "BMI", "DASH score"
        ]][:-8]
        pwmd_demo_df["time disabled"] = pd.to_numeric(pwmd_demo_df["time disabled"].astype(str).strip(), errors='coerce')
        numeric_cols = pwmd_demo_df.select_dtypes(include='number').columns
        pwmd_demo_df[numeric_cols] = pwmd_demo_df[numeric_cols] / 100.0
        pwmd_demo_df["BMI"] = pwmd_demo_df["BMI"] / 70.0
        pwmd_demo_df['Enc_PID'] = PID_encoder.transform(pwmd_demo_df["PID"])

        FULL_pwoutmd_demo_df = pd.read_excel(pwoutmd_xlsx_filepath)
        pwoutmd_demo_df = FULL_pwoutmd_demo_df[[
            "PID", "disability coding", "time disabled", "Actual handedness",
            "What is your age?", "What is your gender?", "BMI", "DASH score"
        ]][:-5]
        pwoutmd_demo_df["time disabled"] = pd.to_numeric(pwoutmd_demo_df["time disabled"].astype(str).strip(), errors='coerce')
        numeric_cols2 = pwoutmd_demo_df.select_dtypes(include='number').columns
        pwoutmd_demo_df[numeric_cols2] = pwoutmd_demo_df[numeric_cols2] / 100.0
        pwoutmd_demo_df["BMI"] = pwoutmd_demo_df["BMI"] / 70.0
        pwoutmd_demo_df = pwoutmd_demo_df[~pwoutmd_demo_df['PID'].isin(['P001', 'P003'])]
        pwoutmd_demo_df['Enc_PID'] = PID_encoder.transform(pwoutmd_demo_df["PID"])

        combined_demo_df = pd.concat([pwmd_demo_df, pwoutmd_demo_df])
        demoENC_df = pd.get_dummies(
            combined_demo_df,
            columns=["disability coding", "Actual handedness", "What is your gender?"],
            drop_first=True
        )
        demoENC_df.drop(columns=["PID"], inplace=True)  # keep Enc_PID only

        full_yX_timeseries_df = pd.concat([metadata_cols_df, ppd_B_X_df], axis=1)

        # Classic fixed splits (still used either for classic loaders OR to form merged pools)
        train_support_df = full_yX_timeseries_df[
            (full_yX_timeseries_df['PID'].isin(train_PIDs)) &
            (full_yX_timeseries_df['Gesture_Num'].isin(train_gesture_range))
        ]
        train_query_df = full_yX_timeseries_df[
            (full_yX_timeseries_df['PID'].isin(train_PIDs)) &
            (full_yX_timeseries_df['Gesture_Num'] == 10)
        ]

        val_support_df = full_yX_timeseries_df[
            (full_yX_timeseries_df['PID'].isin(val_PIDs)) &
            (full_yX_timeseries_df['Gesture_Num'] == 1)
        ]
        val_query_df = full_yX_timeseries_df[
            (full_yX_timeseries_df['PID'].isin(val_PIDs)) &
            (full_yX_timeseries_df['Gesture_Num'].isin(valtest_gesture_range))
        ]

        test_support_df = full_yX_timeseries_df[
            (full_yX_timeseries_df['PID'].isin(test_PIDs)) &
            (full_yX_timeseries_df['Gesture_Num'] == 1)
        ]
        test_query_df = full_yX_timeseries_df[
            (full_yX_timeseries_df['PID'].isin(test_PIDs)) &
            (full_yX_timeseries_df['Gesture_Num'].isin(valtest_gesture_range))
        ]

        emg_cols = [c for c in ppd_B_X_df.columns if c.startswith("EMG")]
        imu_cols = [c for c in ppd_B_X_df.columns if c.startswith("IMU")]
        demo_cols = demoENC_df.columns

        if save_dfs:
            train_support_df.to_pickle(f"{dfs_save_path}{config['timestamp']}_train_support_df.pkl")
            train_query_df.to_pickle(f"{dfs_save_path}{config['timestamp']}_train_query_df.pkl")
            val_support_df.to_pickle(f"{dfs_save_path}{config['timestamp']}_val_support_df.pkl")
            val_query_df.to_pickle(f"{dfs_save_path}{config['timestamp']}_val_query_df.pkl")
            test_support_df.to_pickle(f"{dfs_save_path}{config['timestamp']}_test_support_df.pkl")
            test_query_df.to_pickle(f"{dfs_save_path}{config['timestamp']}_test_query_df.pkl")
            demoENC_df.to_pickle(f"{dfs_save_path}{config['timestamp']}_demoENC_df.pkl")
            with open(f"{dfs_save_path}{config['timestamp']}_columns.pkl", "wb") as f:
                pickle.dump([emg_cols, imu_cols, list(demo_cols)], f)
            print("Dataframes have been saved!")

    else:  # ie if our datasets already exist
        #print(f"dfs_load_path: {dfs_load_path}")
        train_support_df = pd.read_pickle(f"{dfs_load_path}{saved_df_timestamp}_train_support_df.pkl")
        train_query_df = pd.read_pickle(f"{dfs_load_path}{saved_df_timestamp}_train_query_df.pkl")
        val_support_df = pd.read_pickle(f"{dfs_load_path}{saved_df_timestamp}_val_support_df.pkl")
        val_query_df = pd.read_pickle(f"{dfs_load_path}{saved_df_timestamp}_val_query_df.pkl")
        test_support_df = pd.read_pickle(f"{dfs_load_path}{saved_df_timestamp}_test_support_df.pkl")
        test_query_df = pd.read_pickle(f"{dfs_load_path}{saved_df_timestamp}_test_query_df.pkl")
        demoENC_df = pd.read_pickle(f"{dfs_load_path}{saved_df_timestamp}_demoENC_df.pkl")
        with open(f"{dfs_load_path}{saved_df_timestamp}_columns.pkl", "rb") as f:
            emg_cols, imu_cols, demo_cols = pickle.load(f)



def _B_normalize_block(block_np, demean=True, eps=1e-8):
    """
    block_np: (T, D_block) numpy array for one biosignal (e.g., all IMU channels)
    Returns: (T, D_block) normalized per $B (demean per channel, divide by shared std over block)
    """
    if demean:
        block_np = block_np - block_np.mean(axis=0, keepdims=True)  # per-channel demean
    sigma = block_np.ravel().std(dtype=np.float64)
    if sigma < eps:
        return block_np  # flat signal; leave as-is
    return block_np / sigma

def preprocess_df_B_by_gesture(
    data_df: pd.DataFrame,
    biosignal_switch_ix: int = 72,   # [:switch) = IMU, [switch:] = EMG
    trial_length: int = 64,
    demean: bool = True,
    eps: float = 1e-8,
) -> pd.DataFrame:
    """
    Apply $B to every trial in the full dataframe.
    Assumptions:
      - data_df has ONLY sensor columns (no metadata), shape = (num_trials*trial_length, num_channels)
      - IMU columns come first, EMG columns follow
      - Each trial is a contiguous block of `trial_length` rows
    Returns: DataFrame with same shape/columns as input.
    """
    if data_df.isna().any().any():
        print("Warning: NaNs detected in input; consider cleaning first.")

    num_rows, num_cols = data_df.shape
    if num_rows % trial_length != 0:
        raise ValueError(f"Rows ({num_rows}) not divisible by trial_length ({trial_length}).")

    if not (0 < biosignal_switch_ix < num_cols):
        raise ValueError(f"biosignal_switch_ix {biosignal_switch_ix} must be in (0, {num_cols}).")

    num_trials = num_rows // trial_length
    cols = data_df.columns
    X = data_df.to_numpy(dtype=np.float64, copy=True)  # (N, D)

    for t in range(num_trials):
        s = t * trial_length
        e = s + trial_length
        trial = X[s:e, :]  # (T, D)

        imu_block = trial[:, :biosignal_switch_ix]
        emg_block = trial[:, biosignal_switch_ix:]

        imu_block = _B_normalize_block(imu_block, demean=demean, eps=eps)
        emg_block = _B_normalize_block(emg_block, demean=demean, eps=eps)

        X[s:e, :biosignal_switch_ix] = imu_block
        X[s:e, biosignal_switch_ix:] = emg_block

    out = pd.DataFrame(X, columns=cols, index=data_df.index)
    return out

